## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
!nvidia-smi

Thu Dec 12 15:07:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten,Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3), padding = 'same', input_shape = x_train.shape[1:]))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Activation('relu'))
classifier.add(BatchNormalization())
#卷積組合
classifier.add(Convolution2D(32,(3,3)))
classifier.add(Activation('relu'))
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(512)) #output_dim=100,activation=relu
classifier.add(Activation('relu'))
classifier.add(Dropout(0.5))
#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/10
50000/50000 [==============================] - 9s 182us/step - loss: 1.6907 - acc: 0.4424
Epoch 2/10
50000/50000 [==============================] - 5s 103us/step - loss: 1.2112 - acc: 0.5705
Epoch 3/10
50000/50000 [==============================] - 5s 103us/step - loss: 1.0422 - acc: 0.6333
Epoch 4/10
50000/50000 [==============================] - 5s 104us/step - loss: 0.9208 - acc: 0.6774
Epoch 5/10
50000/50000 [==============================] - 5s 104us/step - loss: 0.8111 - acc: 0.7157
Epoch 6/10
50000/50000 [==============================] - 5s 101us/step - loss: 0.7099 - acc: 0.7511
Epoch 7/10
50000/50000 [==============================] - 5s 100us/step - loss: 0.6205 - acc: 0.7817
Epoch 8/10
50000/50000 [==============================] - 5s 101us/step - loss: 0.5434 - acc: 0.8108
Epoch 9/10
50000/50000 [==============================] - 5s 103us/step - loss: 0.4826 - acc: 0.8328
Epoch 10/10
50000/50000 [==============================] - 5s 103us/step - loss: 0.4305 -

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.2759001e-05, 6.0319974e-16, 9.3730509e-05, 9.8232967e-01,
        1.7607436e-08, 1.5156850e-13, 2.0354155e-07, 1.1999172e-15,
        1.7503669e-02, 2.2294938e-15]], dtype=float32)